# Field Guide to Databases

December 2017

# Key:value stores

In [1]:
import dbm
import re

## grab some data

UNIX-based machines generally have a [`words` file](https://en.wikipedia.org/wiki/Words_(Unix)) (used for spellchecking) in `/usr/share/dict/words` or `/usr/dict/words`.  We'll use that as a source of data.

In [2]:
!head /usr/share/dict/words

A
a
aa
aal
aalii
aam
Aani
aardvark
aardwolf
Aaron


In [3]:
import re
vowel_pattern = re.compile('[aeiou]')
raw_data = {}
with open('/usr/share/dict/words') as infile:
    for word in infile.readlines():
        vowels = vowel_pattern.findall(word)
        raw_data[word.rstrip()] = vowels

In [4]:
raw_data['helicopter']

['e', 'i', 'o', 'e']

In [5]:
import dbm
db = dbm.open('vowels.dbm', 'n')

In [6]:
for (k, v) in raw_data.items():
    db[k] = v

TypeError: gdbm mappings have byte or string elements only

In [7]:
for (k, v) in raw_data.items():
    db[k] = str(v)

In [8]:
db['miasma']

b"['i', 'a', 'a']"

In [9]:
eval(db['miasma'])

['i', 'a', 'a']

See also: [shelve](https://docs.python.org/3.1/library/shelve.html)

 # Document db

In [10]:
def word_info(word):
    vowels = vowel_pattern.findall(word)
    result = {
        'raw': word,
        'length': len(word),
        'vowels': vowels,
        'n_vowels': len(vowels),
    }
    return result

In [11]:
word_info('platypus')

{'length': 8, 'n_vowels': 2, 'raw': 'platypus', 'vowels': ['a', 'u']}

## MongoDB

Install MongoDB and create a directory for it to save databases to - by default, `/data/db`.

    % sudo mkdir -p /data/db
    Password:
    ~/Dropbox/arguments % whoami
    catherine
    ~/Dropbox/arguments % sudo chown catherine:everyone /data/db
    ~/Dropbox/arguments % mongod


Start the Mongo server daemon.

    % mongod
    2017-12-12T17:44:13.727-0500 I CONTROL  [initandlisten] MongoDB starting : pid=89611 port=27017 dbpath=/data/db 64-bit host=I05EDJ-2LGFH05.netgear.com
    2017-12-12T17:44:13.728-0500 I CONTROL  [initandlisten] db version v3.4.10
    2017-12-12T17:44:13.728-0500 I CONTROL  [initandlisten] git version: 078f28920cb24de0dd479b5ea6c66c644f6326e9
    2017-12-12T17:44:13.728-0500 I CONTROL  [initandlisten] OpenSSL version: OpenSSL 1.0.2m  2 Nov 2017
    2017-12-12T17:44:13.728-0500 I CONTROL  [initandlisten] allocator: system
    2017-12-12T17:44:13.728-0500 I CONTROL  [initandlisten] modules: none
    2017-12-12T17:44:13.728-0500 I CONTROL  [initandlisten] build environment:
    2017-12-12T17:44:13.729-0500 I CONTROL  [initandlisten]     distarch: x86_64
    2017-12-12T17:44:13.729-0500 I CONTROL  [initandlisten]     target_arch: x86_64
    2017-12-12T17:44:13.729-0500 I CONTROL  [initandlisten] options: {}
    2017-12-12T17:44:13.732-0500 I STORAGE  [initandlisten] exception in initAndListen: 29 Data directory /data/db not found., terminating
    2017-12-12T17:44:13.732-0500 I NETWORK  [initandlisten] shutdown: going to close listening sockets...
    2017-12-12T17:44:13.732-0500 I NETWORK  [initandlisten] shutdown: going to flush diaglog...
    2017-12-12T17:44:13.733-0500 I CONTROL  [initandlisten] now exiting
    2017-12-12T17:44:13.733-0500 I CONTROL  [initandlisten] shutting down with code:100


In [12]:
!pip install pymongo

    100% |████████████████████████████████| 317kB 2.2MB/s ta 0:00:011


In [16]:
import pymongo

In [17]:
>>> from pymongo import MongoClient
>>> client = MongoClient()

In [19]:
db = client.word_db

### everything created lazily

In [20]:
results = db.word_collection.insert(word_info(word) for word in raw_data)

In [23]:
db.word_collection.find_one()

{'_id': ObjectId('5a3063488543df751b9312fd'),
 'length': 18,
 'n_vowels': 9,
 'raw': 'pseudosacrilegious',
 'vowels': ['e', 'u', 'o', 'a', 'i', 'e', 'i', 'o', 'u']}

In [35]:
list(db.word_collection.find({'vowels': ['u']}))[:10]

[{'_id': ObjectId('5a3063488543df751b93130d'),
  'length': 4,
  'n_vowels': 1,
  'raw': 'husk',
  'vowels': ['u']},
 {'_id': ObjectId('5a3063488543df751b931682'),
  'length': 5,
  'n_vowels': 1,
  'raw': 'bushy',
  'vowels': ['u']},
 {'_id': ObjectId('5a3063488543df751b931698'),
  'length': 3,
  'n_vowels': 1,
  'raw': 'cud',
  'vowels': ['u']},
 {'_id': ObjectId('5a3063488543df751b9316eb'),
  'length': 6,
  'n_vowels': 1,
  'raw': 'spunky',
  'vowels': ['u']},
 {'_id': ObjectId('5a3063488543df751b9316fe'),
  'length': 4,
  'n_vowels': 1,
  'raw': 'thud',
  'vowels': ['u']},
 {'_id': ObjectId('5a3063488543df751b9317d9'),
  'length': 5,
  'n_vowels': 1,
  'raw': 'Pyrus',
  'vowels': ['u']},
 {'_id': ObjectId('5a3063488543df751b931809'),
  'length': 6,
  'n_vowels': 1,
  'raw': 'grutch',
  'vowels': ['u']},
 {'_id': ObjectId('5a3063488543df751b931b39'),
  'length': 6,
  'n_vowels': 1,
  'raw': 'scurdy',
  'vowels': ['u']},
 {'_id': ObjectId('5a3063498543df751b931b92'),
  'length': 3,
  '

In [36]:
db.word_collection.find_one({'n_vowels': 0})

{'_id': ObjectId('5a3063488543df751b931509'),
 'length': 5,
 'n_vowels': 0,
 'raw': 'sylph',
 'vowels': []}

In [37]:
db.word_collection.find_one({'vowels': ['i', 'o', 'e', 'a', 'a', 'e']})

{'_id': ObjectId('5a3063498543df751b937f65'),
 'length': 13,
 'n_vowels': 6,
 'raw': 'inconcealable',
 'vowels': ['i', 'o', 'e', 'a', 'a', 'e']}

In [ ]:
list(db.word_collection.find({'vowels': ['i', 'o', 'e', 'a', 'a', 'e']}))

In [ ]:
db.word_collection.drop()

In [ ]:
db.word_collection.count()

In [40]:
db.word_collection.find().count()

471772

In [39]:
list(db.word_collection.find({'n_vowels': {'$gt': 10}}))

[{'_id': ObjectId('5a30634c8543df751b959941'),
  'length': 20,
  'n_vowels': 11,
  'raw': 'palaeometeorological',
  'vowels': ['a', 'a', 'e', 'o', 'e', 'e', 'o', 'o', 'o', 'i', 'a']},
 {'_id': ObjectId('5a30634c8543df751b960154'),
  'length': 23,
  'n_vowels': 11,
  'raw': 'pericardiomediastinitis',
  'vowels': ['e', 'i', 'a', 'i', 'o', 'e', 'i', 'a', 'i', 'i', 'i']},
 {'_id': ObjectId('5a30634c8543df751b962f5a'),
  'length': 21,
  'n_vowels': 11,
  'raw': 'aminoacetophenetidine',
  'vowels': ['a', 'i', 'o', 'a', 'e', 'o', 'e', 'e', 'i', 'i', 'e']},
 {'_id': ObjectId('5a30634c8543df751b9630c8'),
  'length': 22,
  'n_vowels': 11,
  'raw': 'zoologicoarchaeologist',
  'vowels': ['o', 'o', 'o', 'i', 'o', 'a', 'a', 'e', 'o', 'o', 'i']},
 {'_id': ObjectId('5a31d185bc55e10a68a7a9cc'),
  'length': 22,
  'n_vowels': 11,
  'raw': 'zoologicoarchaeologist',
  'vowels': ['o', 'o', 'o', 'i', 'o', 'a', 'a', 'e', 'o', 'o', 'i']},
 {'_id': ObjectId('5a31d187bc55e10a68aa36ef'),
  'length': 21,
  'n_vowe

In [ ]:
%time
db.word_collection.find({'vowels': ['u', 'o']}).count()

In [ ]:
result = db.profiles.create_index([('vowels', pymongo.ASCENDING)],
                                 unique=False)

In [ ]:
%time
db.word_collection.find({'vowels': ['e', 'e']}).count()

In [ ]:
db.drop_collection('word_collection')

# Graph databases: Neo4j

    % brew install neo4j
    % neo4j start
    % open http://localhost:7474